# Smoker Dataset
## Module 10.13 Assignment 10: Final Project
Dec. 2024

Authors: Tyler Earps, Ryan Smith, Basil Mullings, & Ean Vandergraaf

### Stage 0: Abstract and Data Description

### Stage 1: Data Preparation

### Stage 2: Feature Extraction

### Stage 3: Machine Learning Algorithm Preparation and Parameter Tuning

### Stage 4: Model Evaluation

### Stage 5: Visualization

### Limitations, Future Work, and Conclusion

### Overall Performance and Documentation